In [3]:
#Import all the dependencies
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument, TaggedLineDocument
import gzip, os, glob
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# nltk.download('gutenberg')

In [15]:
from os import listdir
from os.path import isfile, join
# LabeledSentence = gensim.models.doc2vec.LabeledSentence
import pandas as pd
import json

In [25]:
path = '/Users/noahedelstein/Desktop/Parsons/design with ml/ml_final/ml_final_python/sotu_umap_position.json'

with open(path) as data_file:
  data = data_file.read()
  # print(data) 
  data_content = json.loads(data)
  # transcripts = [line['transcript'] for line in data_content]
  # print(transcripts[0])
  name = [line['President'] for line in data_content]
  print(name)

['Donald J. Trump', 'Donald J. Trump', 'Barack Obama', 'Barack Obama', 'Barack Obama', 'Barack Obama', 'Barack Obama', 'Barack Obama', 'Barack Obama', 'Barack Obama', 'George W. Bush', 'George W. Bush', 'George W. Bush', 'George W. Bush', 'George W. Bush', 'George W. Bush', 'George W. Bush', 'George W. Bush', 'William J. Clinton', 'William J. Clinton', 'William J. Clinton', 'William J. Clinton', 'William J. Clinton', 'William J. Clinton', 'William J. Clinton', 'William J. Clinton', 'George Bush', 'George Bush', 'George Bush', 'George Bush', 'Ronald Reagan', 'Ronald Reagan', 'Ronald Reagan', 'Ronald Reagan', 'Ronald Reagan', 'Ronald Reagan', 'Ronald Reagan', 'Ronald Reagan', 'Jimmy Carter', 'Jimmy Carter', 'Jimmy Carter', 'Jimmy Carter', 'Jimmy Carter', 'Jimmy Carter', 'Jimmy Carter', 'Gerald R. Ford', 'Gerald R. Ford', 'Gerald R. Ford', 'Richard Nixon', 'Richard Nixon', 'Richard Nixon', 'Richard Nixon', 'Richard Nixon', 'Richard Nixon', 'Richard Nixon', 'Richard Nixon', 'Richard Nixon'

In [21]:
from nltk.corpus import stopwords
import re
import string

In [0]:
stop_words = set(stopwords.words('english'))

tagged_data = []
for i, _d in enumerate(transcripts):
    lowercase = _d.lower()
    remove_urls = re.sub(r"http\S+", "", lowercase)
    remove_punctuation = remove_urls.translate(str.maketrans('','',string.punctuation))
    tokenized = word_tokenize(remove_punctuation)
    words = [w for w in tokenized if not w in stop_words]
    tagged_data.append(TaggedDocument(words=words, tags=[str(i)]))

In [0]:
tagged_data

In [0]:
# tokenizer = RegexpTokenizer(r'/w+')
# stopword_set = set(stopwords.words('english'))

In [0]:
# https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5

max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

In [0]:
model = Doc2Vec.load("d2v.model")

In [0]:
model.wv.most_similar("patriot")

In [0]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [0]:
doc_tags = list(model.docvecs.doctags.keys())
X = model[doc_tags]

In [0]:
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)
df = pd.DataFrame(X_tsne, index=doc_tags, columns=['x', 'y'])

In [0]:
features_extracted = []
for i, review in enumerate(transcripts):
    features_extracted.append(model.docvecs[str(i)])

In [0]:
features_extracted

In [0]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
scaled = ss.fit_transform(features_extracted)

In [0]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10)
clusters = kmeans.fit_predict(scaled)

In [0]:
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(n_neighbors=5, metric='cosine').fit(scaled)
_, closest = neighbors.kneighbors(kmeans.cluster_centers_)

In [0]:
for index, cluster_row in enumerate(closest):
    print('\n\nCluster number ' + str(index) + '\n\n')
    for col in cluster_row:
        print(features_extracted[col])
        print('\n\n')

In [0]:
import umap

embedding = umap.UMAP().fit_transform(scaled)

In [0]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(embedding)
embedding_scaled = scaler.transform(embedding)

In [0]:
embedding_scaled

In [0]:
with open(path) as data_file:
  data = data_file.read()
#   print(data) 
  data_content = json.loads(data)
  presidents = [line['president'] for line in data_content]
  print(president[100])

In [0]:
lookup = []

with open(path) as data_file:
    data = data_file.read()
    data_content = json.loads(data)
    transcripts = [line['transcript'] for line in data_content]
    dates = [line['date'] for line in data_content]
    presidents = [line['president'] for line in data_content]
    for date, president, cluster_pos, transcript in zip(dates, presidents, embedding_scaled, transcripts):
        lookup.append({
            "Date": date,
            "President": president,
            "Cluster_Pos": cluster_pos.tolist(),
            "Speech": transcript
        })

In [0]:
with open('sotu_umap_position.json', 'w') as outfile:
    json.dump(lookup, outfile)